In [1]:
import matplotlib.pyplot as plt
import numpy as np

import pylops
import pyproximal
from scipy.signal import filtfilt, convolve

import warnings
warnings.filterwarnings('ignore')
 
np.random.seed(1)

In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

## An introduction to proximal solvers for engineers
### Day 2 @ Luna Innovation 
### Instructor: Matteo Ravasi, Assistant Professor - KAUST

## Half-Quadratic Splitting (HQS)

When both where $f$ and $g$ are non-smooth, convex functions, the Proximal gradient algorithm (or ISTA) cannot be used.

A two steps procedure is required:

- Splitting: $\mathbf{y}=\mathbf{m}$, such that

$$
\underset{\mathbf{m}, \mathbf{y}} {\mathrm{argmin}} \quad f(\mathbf{m}) + g(\mathbf{y}) \; s.t. \; \mathbf{y}=\mathbf{m}
$$

- Relaxation: $\text{arg} \underset{\mathbf{m},\mathbf{y}} {\mathrm{min}} \mathscr{L}$, where 
$$
\mathscr{L}=f(\mathbf{m}) + g(\mathbf{y}) + \frac{\rho}{2}||\mathbf{m}-\mathbf{y}||_2^2 
$$

## Half-Quadratic Splitting (HQS)

- Alternating minimization:

$$
\mathbf{m}_k = \underset{\mathbf{m}} {\mathrm{argmin}} \; \mathscr{L}(\mathbf{m}, \mathbf{y}_{k-1}) = \underset{\mathbf{m}} {\mathrm{argmin}} \; f(\mathbf{m}) + \frac{\rho_k}{2}||\mathbf{m}-\mathbf{y}_{k-1}||_2^2 = prox_{\tau_k f}(\mathbf{y}_{k-1})
$$
$$
\mathbf{y}_k = \underset{\mathbf{y}} {\mathrm{argmin}} \; \mathscr{L}(\mathbf{m}_k, \mathbf{y}) = \underset{\mathbf{y}} {\mathrm{argmin}} \; g(\mathbf{y}) + \frac{\rho_k}{2}||\mathbf{m}_k-\mathbf{y}||_2^2 =  prox_{\tau_k g}(\mathbf{m}_k)
$$

where $\tau_k=1/\rho_k$. To converge $lim_{k \rightarrow \infty} \rho_k = \infty$ (or $lim_{k \rightarrow \infty} \tau_k = 0$).

## Half-Quadratic Splitting (HQS)

Time to practice: [EX2](https://github.com/mrava87/ProximalTeaching/blob/main/examples/BasisPursuit.ipynb)

## ADMM

Similar to HQS, a two steps procedure is required:

- Splitting: $\mathbf{y}=\mathbf{m}$, such that

$$
\underset{\mathbf{m}, \mathbf{y}} {\mathrm{argmin}} \quad f(\mathbf{m}) + g(\mathbf{y}) \; s.t. \;  \mathbf{y}=\mathbf{m}
$$

- Relaxation by Augmented Lagrangian: $\text{arg} \underset{\mathbf{m},\mathbf{y}} {\mathrm{min}} \underset{\mathbf{z}}  {\mathrm{max}} \mathscr{L}$, where 
$$
\mathscr{L}=f(\mathbf{m}) + g(\mathbf{y}) + \frac{\rho}{2}||\mathbf{m}-\mathbf{y}||_2^2 + \mathbf{z}^T(\mathbf{m}-\mathbf{y})
$$

## ADMM

- Rescaling (scaled form = combine the last two terms):

$$
\frac{\rho}{2}||\mathbf{m}-\mathbf{y}||_2^2 + \mathbf{z}^T(\mathbf{m}-\mathbf{y}) = 
$$
$$
\frac{\rho}{2} ||\mathbf{m}-\mathbf{y} + 1/\rho \mathbf{z}||_2^2 - 1/(2\rho) ||\mathbf{z}||_2^2 = 
$$
$$
\frac{\rho}{2} ||\mathbf{m}-\mathbf{y} + \mathbf{z}'||_2^2 - \frac{\rho}{2} ||\mathbf{z}'||_2^2
$$

where $\mathbf{z}'=\mathbf{z}/\rho$.

## ADMM

So we are left with:

$$
\mathscr{L}=f(\mathbf{m}) + g(\mathbf{y}) + \frac{\rho}{2} ||\mathbf{m}-\mathbf{y} + \mathbf{z}'||_2^2 - \frac{\rho}{2} ||\mathbf{z}'||_2^2
$$


where $\mathbf{z}'=\mathbf{z}/\rho$.

## ADMM

- Alternating minimization:

$$
\mathbf{m}_k = \underset{\mathbf{m}} {\mathrm{argmin}} \quad \mathscr{L}(\mathbf{m}, \mathbf{y}_{k-1}, \mathbf{z}'_{k-1}) = prox_{\tau_k f}(\mathbf{y}_{k-1}-\mathbf{z}'_{k-1})
$$
$$
\mathbf{y}_k = \underset{\mathbf{y}} {\mathrm{argmin}} \quad \mathscr{L}(\mathbf{m}_k, \mathbf{y}, \mathbf{z}'_{k-1}) = prox_{\tau_k g}(\mathbf{m}_k+\mathbf{z}'_{k-1})
$$
$$
\mathbf{z}'_k = \underset{\mathbf{z}'} {\mathrm{argmax}} \quad \mathscr{L}(\mathbf{m}_k, \mathbf{y}_k, \mathbf{z}') = \mathbf{z}'_{k-1} + \mathbf{m}_k - \mathbf{y}_k
$$

where $\tau_k=1/\rho_k \in (0, 1/L]$ with $L$ being the Lipschitz constant of $\nabla f$.

## General ADMM

In theory, the ADMM algorithm is much more powerful than that. It can solve any problem of the form:

$$
\text{arg} \underset{\mathbf{m}} {\mathrm{min}} \quad f(\mathbf{m}) + g(\mathbf{y}) \; s.t. \mathbf{Am}+\mathbf{By}=c
$$
Relaxation by Augmented Lagrangian: $\text{arg} \underset{\mathbf{m},\mathbf{y}} {\mathrm{min}} \underset{\mathbf{z}}  {\mathrm{max}} \mathscr{L}$, where 
$$
\mathscr{L}=f(\mathbf{m}) + g(\mathbf{y}) + \frac{\rho}{2}||\mathbf{Am}+\mathbf{By}-c||_2^2 + \mathbf{z}^T(\mathbf{Am}+\mathbf{By}-c)
$$

## General ADMM

We need to define some special cases to find useful solutions:

1. $\mathbf{A}=\mathbf{I}$, $\mathbf{B}=-\mathbf{I}$, $c=0$: see above.
2. $\mathbf{A}=\mathbf{L}$, $\mathbf{B}=-\mathbf{I}$, $c=0$.

## ADMM (case 2)

When $g(\mathbf{Lm})$ (e.g., $||\mathbf{Lm}||_p$), we follow similar to optimize the associated functional:

$$
\text{arg} \underset{\mathbf{m}} {\mathrm{min}} \quad f(\mathbf{m}) + g(\mathbf{Lm})
$$

- Splitting: $\mathbf{y}=\mathbf{Lm}$
- Augmented Lagrangian: $\text{arg} \underset{\mathbf{m},\mathbf{y}} {\mathrm{min}} \underset{\mathbf{z}}  {\mathrm{max}} \mathscr{L}$, where 
$$
\mathscr{L}=f(\mathbf{m}) + g(\mathbf{y}) + \frac{\rho}{2}||\mathbf{Lm}-\mathbf{y}||_2^2 + \mathbf{z}^T(\mathbf{Lm}-\mathbf{y})
$$

## ADMM (case 2)

- Alternating minimization:

$$
\mathbf{m}_k = \underset{\mathbf{m}} {\mathrm{argmin}} \quad \mathscr{L}(\mathbf{m}, \mathbf{y}_{k-1}, \mathbf{z}_{k-1})
$$
$$
\mathbf{y}_k = \underset{\mathbf{y}} {\mathrm{argmin}} \quad \mathscr{L}(\mathbf{m}_k, \mathbf{y}, \mathbf{z}_{k-1}) = prox_{\tau_k g}(\mathbf{Lm}_{k} + \mathbf{z}_{k-1})
$$
$$
\mathbf{z}_k = \underset{\mathbf{z}} {\mathrm{argmax}} \quad \mathscr{L}(\mathbf{m}_k, \mathbf{y}_k, \mathbf{z}) = \mathbf{z}_{k-1} + \mathbf{Lm}_k - \mathbf{y}_k
$$

where $\tau_k=1/\rho_k \in (0, 1/\lambda_{max}(\mathbf{L}^H \mathbf{L})]$.

## ADMM (case 3)

Let's now consider a special case where

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \quad \frac{1}{2} ||\mathbf{d} - \mathbf{Gm}||_2^2 + g(\mathbf{Lm})
$$

This reduces to:

$$
\mathbf{m}_k = (\mathbf{G}^H \mathbf{G} + \rho_k \mathbf{L}^H \mathbf{L})^{-1} (\mathbf{G}^H \mathbf{d} +  \rho_k \mathbf{L}^H(\mathbf{y}_{k-1} - \mathbf{z}_{k-1}))
$$
$$
\mathbf{y}_k = prox_{\tau_k g}(\mathbf{Lm}_{k} + \mathbf{z}_{k-1}) 
$$
$$
\mathbf{z}_k = \mathbf{z}_{k-1} + \mathbf{Lm}_k - \mathbf{y}_k
$$

## ADMM (case 3)

Note that the first step is the proximal operator of the L2 squared norm of the following system of equations:

$$
\begin{bmatrix} 
\mathbf{G} \\
\sqrt{ \rho_k}\mathbf{L}
  \end{bmatrix} \mathbf{m} = 
\begin{bmatrix} 
\mathbf{d} \\
\sqrt{ \rho_k}(\mathbf{y}_{k-1} - \mathbf{z}_{k-1})
  \end{bmatrix}
$$

and can be solved directly or via any of the iterative scheme for least-squares inversion (e.g., CGLS, LSQR).

## Total-variation (TV) norm

A special norm, commonly used in signal and image processing --> renders a blocky solution.

Two types (here shown for 2D case):

- Anisotropic TV:

$$
TV_a(\mathbf{m})=||\nabla \mathbf{m}||_1 = \sum_{i=1}^{N_xN_z} |\mathbf{m}_x|_i + |\mathbf{m}_z|_i
$$
 
- Isotropic TV:

$$
TV_i(\mathbf{m})=||\nabla \mathbf{m}||_{2,1} = \sum_{i=1}^{N_xN_z} \sqrt{(\mathbf{m}_x)_i^2 +(\mathbf{m}_z)_i^2}
$$

## Total-variation (TV) norm

Anisotropic TV: $TV_a(\mathbf{m})=||\nabla \mathbf{m}||_1 = \sum_{i=1}^{N_xN_z} |\mathbf{m}_x|_i + |\mathbf{m}_z|_i$ 

where $\nabla: \mathbb{R}^{N_xN_z} \rightarrow \mathbb{R}^{2 N_xN_z}$ is the gradient operator, that applies directional derivatives along the x- and z-axes and stacks them vertically: $\nabla \mathbf{m} = [\mathbf{m}_x^T, \mathbf{m}_z^T]^T$.

Can be easily expressed in the form of $g(\mathbf{Lm})$: $g = ||\cdot||_1, \mathbf{L}=\nabla$.

## Total-variation (TV) norm

Isotropic TV: $TV_i(\mathbf{m})=||\nabla \mathbf{m}||_{2,1} = \sum_{i=1}^{N_xN_z} \sqrt{(\mathbf{m}_x)_i^2 +(\mathbf{m}_z)_i^2}$ 

where $\nabla: \mathbb{R}^{N_xN_z} \rightarrow \mathbb{R}^{2 \times  N_xN_z}$ is the gradient operator, that applies directional derivatives along the x- and z-axes and stacks them horizontally: $\nabla \mathbf{m} = [\mathbf{m}_x, \mathbf{m}_z]^T$.

Can be easily expressed in the form of $g(\mathbf{Lm})$: $g = ||\cdot||_{2,1}, \mathbf{L}=\nabla$.

## Total-variation (TV) norm

Let's take now an example:

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \quad \frac{1}{2} ||\mathbf{d} - \mathbf{Gm}||_2^2 + \alpha TV_i(\mathbf{m}) 
$$

This can be seen as a special case of the ADMM (case 3).

## Total-variation (TV) norm

Time to practice: [EX3](https://github.com/mrava87/ProximalTeaching/blob/main/examples/SignalDenoising.ipynb) and [EX4](https://github.com/mrava87/ProximalTeaching/blob/main/examples/MRIReconstruction.ipynb)

## Chambolle-Pock Primal Dual

A more general solver, able to handle the following objective function

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \quad f(\mathbf{m}) + g(\mathbf{Lm})
$$

for any choice of $f$ and $g$ proximable functions, and linear operator $\mathbf{L}$.

**Disclaimer: this is my favourite solver.**

## Chambolle-Pock Primal Dual (PD)

Why PD is my favourite?

- Very flexible in terms of objective function;
- In many problems I work with, outperforms other state-of-the-art solvers.

## Chambolle-Pock Primal Dual (PD)

PD effectively minimizes the equivalent primal-dual problem (primal in $f$, dual in $g$):

$$
\min_{\mathbf{m}} \max_{\mathbf{y}} \mathbf{y}^T(\mathbf{Lm}) + f(\mathbf{m}) - g^*(\mathbf{y})
$$

where $\mathbf{y}$ is the so-called dual variable, and $g^*$ is the so-called convex conjugate function of $g$.

## Chambolle-Pock Primal Dual (PD)

To compute the proximal operator of a convex conjugate function (i.e., dual-proximal), we simply rely on the *Moreau identity*:

$$
\mathbf{x} = prox_{\tau f} (\mathbf{x}) + \tau prox_{\frac{1}{\tau} f^*} \left(\frac{\mathbf{x}}{\tau}\right)
$$

So i we know how to compute $prox_{\tau f}$, we can also compute $prox_{\frac{1}{\tau} f^*}$.

## Chambolle-Pock Primal Dual (PD)

The derivation is rather complicated, I provide here just the algorithm:

$$
\mathbf{z}^{k+1} = prox_{\mu g^*}(\mathbf{y}^{k} + \mu \mathbf{L}\bar{\mathbf{x}}^{k})
$$
$$
\mathbf{x}^{k+1} = prox_{\tau f}(\mathbf{x}^{k} - \tau \mathbf{L}^H \mathbf{z}^{k+1})
$$
$$
\bar{\mathbf{x}}^{k+1} = \mathbf{x}^{k+1} + \theta (\mathbf{x}^{k+1} - \mathbf{x}^k)
$$

where $\tau \mu \lambda_{max}(\mathbf{L}^H\mathbf{L}) < 1$.

## Chambolle-Pock Primal Dual (PD)

Derivation:

- Splitting:
$$
\text{arg} \underset{\mathbf{m}} {\mathrm{min}} \quad f(\mathbf{m}) + g(\mathbf{y}) \; s.t. \mathbf{Lm}=\mathbf{y}
$$

- Relaxation by Lagrangian: $\text{arg} \underset{\mathbf{m},\mathbf{y}} {\mathrm{min}} \underset{\mathbf{z}}  {\mathrm{max}} \mathscr{L}$, where 
$$
\mathscr{L}=f(\mathbf{m}) + g(\mathbf{y}) + \mathbf{z}^T(\mathbf{Lm}-\mathbf{y})
$$

- Use definition of convex conjugate function ($f^{\star}(\mathbf{z}) = \sup_\mathbf{m} \{\mathbf{z}^T\mathbf{m} - f(\mathbf{m})\}$)

$$
\text{arg} \underset{\mathbf{m}} {\mathrm{min}} \underset{\mathbf{z}}  {\mathrm{max}} \; \mathbf{z}^T\mathbf{Lm} + f(\mathbf{m}) - g^{\star}(\mathbf{z})
$$

## Chambolle-Pock Primal Dual (PD)

Derivation:

- Define optimality conditions:
$$
0 \in -\partial g^{\star}(\mathbf{z}) + \mathbf{Lm}
$$
$$
0 \in \partial f(\mathbf{m}) + \mathbf{L}^T\mathbf{z}
$$

- Adding $\mathbf{m}$ on both sides:
$$
(\mathbf{I} + \partial g^{\star})(\mathbf{z}) \in  \mathbf{z} + \mathbf{L}\mathbf{m}
$$
$$
(\mathbf{I} + \partial f)(\mathbf{m}) \in  \mathbf{m} - \mathbf{L}^T\mathbf{z}
$$

- Invoking the definition of proximal

$$
\mathbf{z} \in (\mathbf{I} + \partial g^{\star})^{-1}(\mathbf{z} + \mathbf{L}\mathbf{m}) = prox_{g^{\star}}(\mathbf{z} + \mathbf{L}\mathbf{m})
$$
$$
\mathbf{m} \in  (\mathbf{I} + \partial f)^{-1}(\mathbf{m} - \mathbf{L}^T\mathbf{z}) = prox_f(\mathbf{m} - \mathbf{L}^T\mathbf{z})
$$

## Indicator functions

Useful to introduce hard-constraints in optimization.

Definition: a function that maps elements of the subset to one, and all other elements to zero ($I: M \rightarrow {0, 1}$)

$$
I_A(\mathbf{m}) = [\mathbf{m} \in A] = \begin{cases}
1 \quad \mathbf{m} \in A\\
0 \quad \mathbf{m} \notin A
\end{cases}
$$

The proximal operator corresponds to its orthogonal projection.

## Indicator functions

Examples: 

- Box: $\operatorname{Box}_{[l, u]}(m) = \{ m: l \leq x\leq u \}$
  
- Simplex: $\Delta_r(\mathbf{m}) = \{ \mathbf{m}: \sum_i m_i = r,\; m_i \geq 0 \}$

- $L_0$ Ball: $L0_r(\mathbf{m}) = \{ \mathbf{m}: ||\mathbf{m}||_0 \leq r \}$

- $L_1$ Ball: $L1_r(\mathbf{m}) = \{ \mathbf{m}: ||\mathbf{m}||_1 \leq r \}$

- Euclidean Ball: $\operatorname{Eucl}_{[\mathbf{c}, r]} = \{ \mathbf{m}: l ||\mathbf{m} - \mathbf{c}||_2 \leq r \}$

- ...

## Indicator functions

Examples: 

- Box: 
$$
P_{\operatorname{Box}_{[l, u]}} (m) = min\{ max \{m, l\}, u \} =
        \begin{cases}
        l, & m < l\\
        m,& l \leq m_i \leq u \\
        u,  & m > u\\
        \end{cases}
$$

- Simplex:
$$
P_{Box_{[0, \infty]} \cap H_{\mathbf{1}, r}} (\mathbf{m}) = P_{Box_{[0, \infty]}} (\mathbf{x} - \mu^* \mathbf{1})
$$
$\mu^*$ obtained by solving the following equation by bisection: $f(\mu) = \mathbf{1}^T P_{Box_{[0, \infty]}} (\mathbf{m} - \mu \mathbf{c}) - r$.


## Indicator functions

Examples: 

- $L_0$ Ball: retain the $r$ highest (in absolute value) largest entries of $\mathbf{m}$ and zero all the other entries.

- $L_1$ Ball: 
$$
P_{L1_{r}} (\mathbf{m}) = sign(\mathbf{m}) P_{\operatorname{Simplex}(r)}(\mathbf{m})
$$

- Euclidean Ball: 
$$
P_{\operatorname{Eucl}_{[\mathbf{c}, r]}} (\mathbf{m}) = \mathbf{c} + \frac{r}
        {max\{ ||\mathbf{m} - \mathbf{c}||_2^2, r\}}(\mathbf{m} - \mathbf{c})
$$

## Indicator functions

The algorithms that we have discussed up until now can be easily extended to include indicator functions. 

Examples:

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \; \|\mathbf{m}\|_1 \; \text{s.t.} \;  \|\mathbf{Gm} - \mathbf{d}\|_2 < \epsilon
$$

which we can equivalently write as 

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \; \|\mathbf{m}\|_1 + i_{\|\mathbf{Gm} - \mathbf{d}\|_2 < \epsilon}
$$

## Nonlinear constrained inversion

The algorithms that we have discussed up until now can also be extended to nonlinear problems. 

Examples:

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \quad f(\mathbf{m}) \quad s.t. \quad \mathbf{m}
        \in I_{Box}
$$

or

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \quad f(\mathbf{m}) + \alpha TV_i(\mathbf{m}) 
$$

## Nonlinear constrained inversion

- Proximal gradient:

$$
\mathbf{m}_k = prox_{\delta I_{Box}} (\mathbf{m}_{k-1} - \nabla f(\mathbf{m}_{k-1}))\\
$$

- ADMM:

$$
\mathbf{m}_k = prox_{\tau_k f}(\mathbf{y}_{k-1}-\mathbf{z}'_{k-1}) = \underset{\mathbf{m}} {\mathrm{argmin}} \quad f(\mathbf{m}) + \frac{1}{2\tau_k} ||\mathbf{m} - \mathbf{y}_{k-1} + \mathbf{z}'_{k-1}||_2^2
$$
$$
\mathbf{y}_k =  prox_{\delta I_{Box}} (\mathbf{m}_k+\mathbf{z}'_{k-1})
$$
$$
\mathbf{z}'_k = \mathbf{z}'_{k-1} + \mathbf{m}_k - \mathbf{y}_k
$$

## Nonlinear constrained inversion

Time to practice: [EX5](https://github.com/mrava87/ProximalTeaching/blob/main/examples/Nonlinear.ipynb)

## Prior stacking

Most of the algorithms discussed so far can be adapted to scenarios with multiple priors (i.e., multiple $g$ functions):

$$
\underset{\mathbf{m}} {\mathrm{argmin}} \quad f(\mathbf{m}) + \sum_i g_i(\mathbf{m})
$$

- for ADMM: simply perform multiple splittings ($\mathbf{y}_i=\mathbf{m}$)

$$
\underset{\mathbf{m}, \mathbf{y}_1, \mathbf{y}_2, ..., \mathbf{y}_N} {\mathrm{argmin}} \quad f(\mathbf{m}) + \sum_i g_i(\mathbf{y}_i) \; s.t. \;  \mathbf{y}_1=\mathbf{m}, \mathbf{y}_2=\mathbf{m}, ..., \mathbf{y}_N=\mathbf{m}
$$

$$
\underset{\mathbf{m}, \mathbf{y}_1, \mathbf{y}_2, ..., \mathbf{y}_N} {\mathrm{argmin}} \underset{\mathbf{z}_1, \mathbf{z}_2, ..., \mathbf{z}_N}  {\mathrm{max}} f(\mathbf{m}) + \sum_i \left[ g_i(\mathbf{y}_i) + \frac{\rho_i}{2} ||\mathbf{m}-\mathbf{y}_i + \mathbf{z}_i||_2^2 - \frac{\rho_i}{2} ||\mathbf{z}_i||_2^2\right]
$$


## Prior stacking

$$
\mathbf{m}_k = \underset{\mathbf{m}} {\mathrm{argmin}} \quad \mathscr{L}(\mathbf{m}, \mathbf{y}_{1,k-1},...,\mathbf{y}_{N,k-1}, \mathbf{z}_{1,k-1}, \mathbf{z}_{N,k-1}) = ^*
$$
$$
\left\{
    \begin {aligned}
         & \mathbf{y}_{1,k} = \underset{\mathbf{y}_1} {\mathrm{argmin}} \quad \mathscr{L}(\mathbf{m}_k, \mathbf{y}_{1},...,\mathbf{y}_{N,k-1}, \mathbf{z}_{1,k-1}, \mathbf{z}_{N,k-1}) = prox_{\tau_k g_1}(\mathbf{m}_k+\mathbf{z}_{1,k-1})\\
  & ...       \\
  \\
         & \mathbf{y}_{N,k} = \underset{\mathbf{y}_N} {\mathrm{argmin}} \quad \mathscr{L}(\mathbf{m}_k, \mathbf{y}_{1,k},...,\mathbf{y}_{N}, \mathbf{z}_{1,k-1}, \mathbf{z}_{N,k-1}) = prox_{\tau_k g_N}(\mathbf{m}_k+\mathbf{z}_{N,k-1})\\             
    \end{aligned}
\right.
$$
$$
\left\{
    \begin {aligned}
         & \mathbf{z}_{1,k} = \mathbf{z}_{1,k-1} + \mathbf{m}_k - \mathbf{y}_{1,k}\\
  & ...       \\
  \\
         & \mathbf{z}_{N,k} = \mathbf{z}_{N,k-1} + \mathbf{m}_k - \mathbf{y}_{N,k}\\             
    \end{aligned}
\right.
$$

## Prior stacking

$^*$ when $f(\mathbf{m}) = \frac{1}{2} ||\mathbf{d} - \mathbf{Gm}||_2^2$, then:
$$
\mathscr{L}=\frac{1}{2} ||\mathbf{d} - \mathbf{Gm}||_2^2 + \sum  \frac{\rho_i}{2} ||\mathbf{m}-\mathbf{y}_i + \mathbf{z}_i||_2^2 \rightarrow 
\begin{bmatrix} 
\mathbf{G} \\
\sqrt{\rho_1}\mathbf{I}\\
...\\
\sqrt{ \rho_N}\mathbf{I}\\
  \end{bmatrix} \mathbf{m} = 
\begin{bmatrix} 
\mathbf{d} \\
\sqrt{ \rho_1}(\mathbf{y}_{1} - \mathbf{z}_{N}) \\
...\\
\sqrt{ \rho_N}(\mathbf{y}_{N} - \mathbf{z}_{N})
  \end{bmatrix}
$$

$^*$ when $f(\mathbf{m})$ is nonlinear, simply solve the following with numerical methods:
$$
\mathscr{L}=\frac{1}{2} f(\mathbf{m}) + \sum_i  \frac{\rho_i}{2} ||\mathbf{m}-\mathbf{y}_i + \mathbf{z}_i||_2^2
$$

## Prior stacking

Most of the algorithms discussed so far can be adapted to scenarios with multiple priors (i.e., multiple $g$ functions):

- for ADMM with $\mathbf{L}$/PD: 

$$
\mathbf{y} = \begin{bmatrix} 
\mathbf{y}_1 \\
\mathbf{y}_2 \\
... \\
\mathbf{y}_N \\
  \end{bmatrix} =  
\begin{bmatrix} 
\mathbf{L}_1\mathbf{m} \\
\mathbf{L}_2\mathbf{m} \\
... \\
\mathbf{L}_N\mathbf{m}\\
  \end{bmatrix} \rightarrow g(\mathbf{y}) : g_i(\mathbf{y}_i) = g_i(\mathbf{L}_i\mathbf{m})
$$

and use property of proximal of sum of separable functions:

$$
prox_{\tau g} (\mathbf{y}) = [prox_{\tau g_1} (\mathbf{y}_1), prox_{\tau g_2} (\mathbf{y}_2), ..., prox_{\tau g_N} (\mathbf{y}_N)]^T
$$

## Plug-and-Play Priors (PnP)

Let's consider an alternative view of the proximal operator of a generic function $f$:

$$
prox_{\tau g} (\mathbf{v})= \underset{\mathbf{m}} {\mathrm{argmin}} \; \frac{1}{2\tau}||\mathbf{v}-\mathbf{m}||_2^2 + g({\mathbf{m}})
$$

Assuming that $\mathbf{v}$ is a noisy observation (with uncorrelated Gaussian noise), this optimization problem is equivalent to a Maximum a Posterior (MAP) denoising problem with $g$ being the regularizer. 

## Plug-and-Play Priors (PnP)

We can see this is we define $\mathbf{v} = \mathbf{m} + \mathbf{n}$ with $\mathbf{n}\sim \mathcal{N}(0, \sigma_v \mathbf{I})$, such that:

$$
p(\mathbf{v}|\mathbf{m}) \alpha e^{-\frac{||\mathbf{v}-\mathbf{m}||_2^2}{2\sigma_v^2}},
$$
$$p(\mathbf{m}) \alpha e^{-g(\mathbf{m})}
$$

The MAP estimator ($\mathbf{m}_{MAP} = \underset{\mathbf{m}} {\mathrm{argmax}} \quad p(\mathbf{v}|\mathbf{m})p(\mathbf{m})$) becomes:
$$
\mathbf{m}_{MAP} = \underset{\mathbf{m}} {\mathrm{argmin}} \; -log p(\mathbf{v}|\mathbf{m}) - log p(\mathbf{m}) = \underset{\mathbf{m}} {\mathrm{argmin}} \;  \frac{1}{2\sigma_v^2}||\mathbf{v}-\mathbf{m}||_2^2 + g(\mathbf{m})
$$

## Plug-and-Play Priors (PnP)

So we can decide to replace the proximal of a generic function $f$ with any MAP denoiser of choice (or, in practice, even a denoiser that optimizes a different criterion, e.g. MMSE)

$$
prox_{\tau g} (\mathbf{v})= Denoiser(\mathbf{v}, \tau)
$$

This is very popular nowadays with Neural Networks!

## Plug-and-Play Priors (PnP)

Finally, let's also re-interpret the original problem in a Bayesian setting and look for its MAP estimator. For a linear data misfit term:

$$
\mathbf{m}_{MAP} = \underset{\mathbf{m}} {\mathrm{argmin}} \; -log p(\mathbf{d}|\mathbf{m}) - log p(\mathbf{m}) = \underset{\mathbf{m}} {\mathrm{argmin}} \;  \frac{1}{2\sigma_d^2} ||\mathbf{d} - \mathbf{Gm}||_2^2 + g(\mathbf{m})
$$

Note that the prior can be implicitely learned by training a denoiser on a set of representative solutions of the problem.

## Plug-and-Play Priors (PnP)

Time to practice: [EX6](https://github.com/mrava87/ProximalTeaching/blob/main/examples/MRIPnP.ipynb)

## Conclusions

- Proximal algorithms are very powerful and versatile: they can handle all sort of smooth and non-smooth objective functions;

- At the core of every proximal algorithm lie: i) gradient and ii) proximal operator of the different terms of the objective function (plus dual-proximal operator for primal-dual algorithms);

- Do not use proximal algorithms if your objective function is smooth (just rely on gradient based methods, first- or second-order);

- Recent connections between Bayesian formalism and proximal algorithms has shown that one can use denoisers to implicitely capture the prior probability of the solution (use denoising Neural Networks provided availability of representative training data).